In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [2]:
import numpy as np
import pandas as pd
import warnings

In [3]:
warnings.filterwarnings('ignore')

# Get The Dataset

In [4]:
column_names=["user_id","item_id","rating","timestamp"]
df=pd.read_csv("u.data",sep='\t',names=column_names)

In [5]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
df['user_id'].nunique()

943

In [7]:
df['item_id'].nunique()

1682

In [8]:
movies_titles=pd.read_csv('u.item', sep='|', encoding='latin-1',header=None)

In [9]:
movies_titles=movies_titles[[0,1]]

In [10]:
movies_titles.head()

,0,1
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [11]:
movies_titles.columns=['item_id','title']

In [12]:
movies_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [13]:
df=pd.merge(df,movies_titles,on="item_id")

In [14]:
df.tail()

,user_id,item_id,rating,timestamp,title
99995,840,1674,4,891211682,Mamma Roma (1962)
99996,655,1640,3,888474646,"Eighth Day, The (1996)"
99997,655,1637,3,888984255,Girls Town (1996)
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1..."
99999,655,1641,3,887427810,Dadetown (1995)


# Exploratory Data Analysis

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
df.groupby('title').mean()['rating'].sort_values(ascending=False)

In [ ]:
df.groupby('title').count()['rating'].sort_values(ascending=False)

In [ ]:
ratings=pd.DataFrame(df.groupby('title').mean()['rating'])

In [ ]:
ratings.head()

In [ ]:
ratings['num of ratings']=pd.DataFrame(df.groupby('title').count()['rating'])

In [ ]:
ratings

In [ ]:
ratings.sort_values(by="rating",ascending=False)

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(ratings["num of ratings"],bins=70)
plt.show()

In [ ]:
plt.hist(ratings["rating"],bins=70)
plt.show()

In [ ]:
sns.jointplot(x='rating',y='num of ratings',data=ratings,alpha=0.6)

# Creating Recommendations

In [ ]:
df.head()

In [ ]:
moviemat=df.pivot_table(index="user_id",columns="title",values="rating")

In [ ]:
moviemat

In [ ]:
ratings.sort_values('num of ratings',ascending=False).head()

In [ ]:
starwars_user_ratings=moviemat['Star Wars (1977)']
starwars_user_ratings

In [ ]:
similar_to_starwars=moviemat.corrwith(starwars_user_ratings)
similar_to_starwars

In [ ]:
corr_starwars=pd.DataFrame(similar_to_starwars,columns=['Correlation'])
corr_starwars

In [ ]:
corr_starwars.dropna(inplace=True)

In [ ]:
corr_starwars.head()

In [ ]:
corr_starwars.sort_values('Correlation',ascending=False)

In [ ]:
corr_starwars=corr_starwars.join(ratings['num of ratings'])
corr_starwars.head()

In [ ]:
corr_starwars[corr_starwars['num of ratings']>100].sort_values('Correlation',ascending=False)

# Predict Function

In [ ]:
def predict_movies(movie_name):
    movie_user_ratings=moviemat[movie_name]
    similar_to_movie=moviemat.corrwith(movie_user_ratings)
    corr_movie=pd.DataFrame(similar_to_movie,columns=['Correlation'])
    corr_movie.dropna(inplace=True)
    corr_movie=corr_movie.join(ratings['num of ratings'])
    predictions=corr_movie[corr_movie['num of ratings']>100].sort_values('Correlation',ascending=False)
    return predictions

In [ ]:
predictions=predict_movies("Titanic (1997)")
predictions.head()